In [ ]:
import nltk
from nltk import word_tokenize
import string

#nc = 'utf-8'
#ile = open('finalCleaned', 'r',encoding=enc)
#ale_text = file.read()
enc ='utf-8'
file =open('finalCleaned' ,'r' ,encoding=enc)
sample_text =file.read()
#ample_text ='fintech pragmatic assessment disruptive potential financial services future financial services series prepared collaboration deloitte august 2017 foreword consistent economic forum'

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = filter(lambda x: x in printable, text) #filter funny characters, if any.
    text = "".join(list(text))
    return text

Cleaned_text = clean(sample_text)

text = word_tokenize(Cleaned_text)

print ("Tokenized Text: \n")
print (text)


POS_tag = nltk.pos_tag(text)

print ("Tokenized Text with POS tags: \n")
print (POS_tag)



from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
#rint (emmatized_text)
print (lemmatized_text)

POS_tag = nltk.pos_tag(lemmatized_text)

print ("Lemmatized text with POS tags: \n")
print (POS_tag)

stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

stopword_file = open ("stopword_file" ,"r")
#topword_file = open("long_stopwords.txt", "r")
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)
                      
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print( processed_text)
                      
vocabulary = list(set(processed_text))
print( vocabulary)

import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])
                        
                        
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]
        
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in xrange(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in xrange(0,vocab_len):
        
        summation = 0
        for j in xrange(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print( "Converging at iteration "+str(iter)+"....")
        break
        
for i in xrange(0,vocab_len):
    print( "Score of "+vocabulary[i]+": "+str(score[i]))

Tokenized Text: 

['fintech', 'pragmatic', 'assessment', 'disruptive', 'potential', 'financial', 'services', 'future', 'financial', 'services', 'series', 'prepared', 'collaboration', 'deloitte', 'august', '2017', 'foreword', 'consistent', 'economic', 'forum', 'mission', 'applying', 'multistakeholder', 'approach', 'address', 'issues', 'global', 'impact', 'creating', 'report', 'involved', 'extensive', 'outreach', 'dialogue', 'numerous', 'organizations', 'individuals', 'included', 'forum', "'s", 'financial', 'services', 'innovation', 'technology', 'communities', 'professionals', 'academia', 'public', 'sector', 'outreach', 'involved', '150', 'interviews', 'international', 'workshop', 'sessions', 'encouraging', 'collaborative', 'dialogue', 'discuss', 'insights', 'opportunities', 'concerning', 'fintech', 'disruption', 'financial', 'services', 'industry', 'holistic', 'global', 'perspective', 'report', 'enriched', 'support', 'contributions', 'subject', 'matter', 'experts', 'assisted', 'driving

Tokenized Text with POS tags: 

[('fintech', 'JJ'), ('pragmatic', 'JJ'), ('assessment', 'NN'), ('disruptive', 'JJ'), ('potential', 'JJ'), ('financial', 'JJ'), ('services', 'NNS'), ('future', 'VBP'), ('financial', 'JJ'), ('services', 'NNS'), ('series', 'NN'), ('prepared', 'VBN'), ('collaboration', 'NN'), ('deloitte', 'NN'), ('august', 'NN'), ('2017', 'CD'), ('foreword', 'NN'), ('consistent', 'JJ'), ('economic', 'JJ'), ('forum', 'NN'), ('mission', 'NN'), ('applying', 'VBG'), ('multistakeholder', 'JJR'), ('approach', 'NN'), ('address', 'NN'), ('issues', 'NNS'), ('global', 'JJ'), ('impact', 'NN'), ('creating', 'VBG'), ('report', 'NN'), ('involved', 'VBN'), ('extensive', 'JJ'), ('outreach', 'NN'), ('dialogue', 'NN'), ('numerous', 'JJ'), ('organizations', 'NNS'), ('individuals', 'NNS'), ('included', 'VBD'), ('forum', 'NN'), ("'s", 'POS'), ('financial', 'JJ'), ('services', 'NNS'), ('innovation', 'NN'), ('technology', 'NN'), ('communities', 'NNS'), ('professionals', 'NNS'), ('academia', 'VBP')

Lemmatized text with POS tags: 

[('fintech', 'JJ'), ('pragmatic', 'JJ'), ('assessment', 'NN'), ('disruptive', 'JJ'), ('potential', 'JJ'), ('financial', 'JJ'), ('service', 'NN'), ('future', 'NN'), ('financial', 'JJ'), ('service', 'NN'), ('series', 'NN'), ('prepared', 'VBN'), ('collaboration', 'NN'), ('deloitte', 'NN'), ('august', 'NN'), ('2017', 'CD'), ('foreword', 'NN'), ('consistent', 'JJ'), ('economic', 'JJ'), ('forum', 'NN'), ('mission', 'NN'), ('applying', 'VBG'), ('multistakeholder', 'JJR'), ('approach', 'NN'), ('address', 'NN'), ('issue', 'NN'), ('global', 'JJ'), ('impact', 'NN'), ('creating', 'VBG'), ('report', 'NN'), ('involved', 'VBN'), ('extensive', 'JJ'), ('outreach', 'NN'), ('dialogue', 'NN'), ('numerous', 'JJ'), ('organization', 'NN'), ('individual', 'NN'), ('included', 'VBD'), ('forum', 'NN'), ("'s", 'POS'), ('financial', 'JJ'), ('service', 'NN'), ('innovation', 'NN'), ('technology', 'NN'), ('community', 'NN'), ('professional', 'JJ'), ('academia', 'NN'), ('public', 'NN')